## **Set Biobliometric Data for Scopus**

### Clean Data

In [ ]:
import pandas as pd
import re

In [ ]:
#Definir ruta del archivo
input_csv = 'Search_criteria/scopus_MFC_Bibliometric.csv' 

def clean_scopus_data(input_csv, scopus_cleaned):
    # Cargar datos
    df = pd.read_csv(input_csv, encoding='utf-8')
    
    # Normalizar nombres de columnas para evitar errores
    #df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    
    # Eliminar duplicados basados en título y DOI
    df = df.drop_duplicates(subset=['Title', 'DOI'], keep='first')
    
    # Eliminar registros sin título, DOI o palabras clave del autor
    df = df.dropna(subset=['Title', 'DOI', 'Author Keywords', 'Index Keywords' ])
    
    # Función para limpiar texto dentro de <>
    def clean_text(text):
        return re.sub(r'<.*?>', '', str(text))
    
    # Aplicar limpieza a todas las columnas de texto
    df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)
        
    # Guardar CSV limpio
    df.to_csv(scopus_cleaned, index=False)
    
    print(f"✅ Datos limpios guardados en {scopus_cleaned} con {len(df)} registros.")

# Ejemplo de uso:
# clean_scopus_data('scopus_raw.csv', 'scopus_cleaned.csv', start_year=2015)
clean_scopus_data(input_csv,'Search_criteria/scopus_cleaned.csv')

### Modify Data for Bibliometrix

In [ ]:
#Cleaned data
archivo_csv = 'Search_criteria/scopus_cleaned.csv'
df = pd.read_csv(archivo_csv)

# Verificar si la columna "JI" existe, si no, agregarla vacía
if "JI" not in df.columns:
    df["JI"] = ""

# Verificar si la columna "WC" existe, si no, agregarla vacía
if "WC" not in df.columns:
    df["WC"] = ""

# 🔹 Renombrar columnas a las esperadas por Biblioshiny
columnas_esperadas = {
    "Authors": "AU",
    "Title": "TI",
    "DOI": "DI",
    "Source title": "SO",
    "Year": "PY",
    "Cited by": "TC",
    "Affiliations": "C1",
    "Author Keywords": "DE",
    "Index Keywords": "ID",
    "Abstract": "AB",
    "Correspondence Address": "RP",
    "References": "CR"
}

# Solo renombrar si las columnas existen en el archivo
df.rename(columns={col: columnas_esperadas[col] for col in df.columns if col in columnas_esperadas}, inplace=True)

# 🔹 Guardar el nuevo archivo en el formato correcto
archivo_salida = "Search_criteria/formateado_biblioshiny.csv"
df.to_csv(archivo_salida, index=False)

print(f"✅ Archivo formateado guardado como: {archivo_salida}")

### Extract countries

In [ ]:
import re

archivo_csv = 'Search_criteria/formateado_biblioshiny.csv'
df = pd.read_csv(archivo_csv)


# Función para extraer los países de la columna "C1"
def extract_countries(affiliations):
    if pd.isna(affiliations):
        return []
    # Suponiendo que los países están al final de cada afiliación, separados por comas
    countries = re.findall(r',\s*([A-Za-z\s]+)$', affiliations, re.MULTILINE)
    return countries

# Aplicar la función para extraer países
df["Countries"] = df["C1"].apply(extract_countries)

# Seleccionar solo las columnas de autores y países
df_countries = df[["AU", "Countries"]]

# Mostrar algunas filas del resultado
#df_countries.head()

df.to_csv('df_countries.csv', index=False)